In [ ]:
# !pip install pandas transformers torch underthesea vncorenlp tqdm hf_xet

In [ ]:
import json
import numpy as np
import pandas as pd
import torch
import math
from tqdm import tqdm
from underthesea import word_tokenize, pos_tag
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
from sklearn.decomposition import TruncatedSVD

#### Perplexity là gì?

**Perplexity (PPL)** là một chỉ số đo độ “bối rối” của mô hình ngôn ngữ khi dự đoán từ tiếp theo trong câu.  

Nói cách khác: **mức độ khó dự đoán một chuỗi văn bản**.  
Công thức cơ bản cho một câu w_1, w_2, ..., w_N  với mô hình LM:

![alt text](image.png)

- *PPL >=1* 
- *PPL = 1* → câu quá dễ, mô hình dự đoán hoàn hảo (hiếm xảy ra).
- *PPL cao* → câu khó dự đoán, nhiều khả năng từ vựng khác nhau → độ phức tạp cao..


In [ ]:
def extract_linguistic_features(sentence, connectors, lit_terms):
    tokens = word_tokenize(sentence)
    sentence_length = len(tokens)
    avg_word_length = np.mean([len(t) for t in tokens]) if tokens else 0

    pos = pos_tag(sentence)
    num_nouns = sum(1 for _, p in pos if p.startswith("N"))
    num_verbs = sum(1 for _, p in pos if p.startswith("V"))
    num_adjs = sum(1 for _, p in pos if p.startswith("A"))
    num_lit_terms = sum(1 for t in tokens if t in lit_terms)
    num_clauses = sum(1 for t in tokens if t.lower() in connectors)
    num_punct = sum(1 for t in tokens if t in ".,;!?")
    

    return [sentence_length, avg_word_length, num_clauses, num_punct,
            num_nouns, num_verbs, num_adjs, num_lit_terms]

In [ ]:
def get_embedding(sentence,phobert_tokenizer,phobert_model,device, max_length=256):
    inputs = phobert_tokenizer(sentence, return_tensors="pt", truncation=True,
                               padding=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = phobert_model(**inputs)
    emb = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
    return emb

In [ ]:
def parse_tree_depth(sentence, nlp):
    if not sentence or not sentence.strip():
        return 0
    
    parsed = nlp(sentence)
    max_depth = 0

    # duyệt qua từng token
    for sent in parsed.sentences:
        for word in sent.words:
            depth = 1
            head = word.head
            # leo lên cây từ token tới gốc
            while head != 0:
                parent = next(w for w in sent.words if w.id == head)
                head = parent.head
                depth += 1
            max_depth = max(max_depth, depth)

    return max_depth


### Feature đánh giá độ phức tạp câu

##### Feature thống kê cú pháp (syntactic/structural features)

| Feature | Kiểu | Mô tả | Ví dụ |
|---------|------|-------|-------|
| sentence_length | Numeric | Số từ trong câu | 13 |
| avg_word_length | Numeric | Độ dài trung bình của từ | 3.3 |
| num_clauses | Numeric | Số mệnh đề ước lượng bằng liên từ | 0 |
| num_punct | Numeric | Số dấu câu | 0 |
| num_nouns | Numeric | Số danh từ (`N*`) | 8 |
| num_verbs | Numeric | Số động từ (`V*`) | 2 |
| num_adjs | Numeric | Số tính từ (`A*`) | 0 |


In [ ]:
def compute_perplexity(sentence,gpt2_tokenizer, gpt2_model,device, max_length=256):
    encodings = gpt2_tokenizer(sentence, return_tensors="pt",
                               truncation=True, max_length=max_length).to(device)
    with torch.no_grad():
        outputs = gpt2_model(**encodings, labels=encodings["input_ids"])
    return torch.exp(outputs.loss).item()

In [ ]:
def run_sen_comlex(input_file, output_file, connectors,phobert_tokenizer, phobert_model
                   , gpt2_tokenizer, gpt2_model, device, lit_terms, svd, nlp):
    with open(input_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    rows = []
    embs = []
    meta = []

    for item in tqdm(data, desc="Processing questions"):
        qid = item["id"]
        subject = item["subject"]
        question = item["question"]

        # Feature ngôn ngữ học
        ling_feats = extract_linguistic_features(question, connectors, lit_terms)
        # Độ sâu cây cú pháp
        max_depth = parse_tree_depth(question, nlp)
        ling_feats.append(max_depth)
        # Embedding PhoBERT
        emb = get_embedding(question,phobert_tokenizer, phobert_model,device)
        # Perplexity GPT2
        ppl = compute_perplexity(question, gpt2_tokenizer, gpt2_model, device)

        meta.append([qid, subject, question] + ling_feats + [ppl])
        embs.append(emb)

    embs = np.array(embs)
    reduced_embs = svd.fit_transform(embs)
    for i in range(len(data)):
        rows.append(meta[i] + reduced_embs[i].tolist())
    # ======================
    # Xuất CSV
    # ======================
    emb_dim = len(rows[0]) - 3 - 9 - 1  # trừ id, subject, question, 7 feature ngôn ngữ, ppl
    columns = ["id", "subject", "question",
            "sentence_length", "avg_word_length", "num_clauses", "num_punct",
            "num_nouns", "num_verbs", "num_adjs","num_lit_terms","parse_tree_depth", "perplexity"] + [f"emb_{i}" for i in range(emb_dim)]

    df = pd.DataFrame(rows, columns=columns)
    df.to_csv(output_file, index=False, encoding="utf-8-sig")

    print("✅ Đã xuất độ phức tạp của câu hỏi sang file CSV:", output_file)
    print(df.head())